# ARQUITECTURA PARA EL PRODUCTO DE DATOS

![Diagrama arquitectura](propuesta_arquitectura\diagrama_arquitectura.png)



**nota:** si el diagrama no se visualiza por alguna razon, capaz el tamaño. Porfavor visualizarlo en la carpeta propuesta_arquitectura de este repositorio. Alla esta guardado en formato .pdf y .png


### **Validacion marcación cliente con malas practicas transaccionales**
1. El usuario hace una nueva transaccion por medio del servicio transaccional (app)
2. El servicio transaccional revisa la BD Redis de clientes para saber si tiene marcación de malas practicas vigente (< a un dia/24 horas de generación desde la fecha y hora de la transaccion actual)
3. Si el cliente tiene marcacion vigente el sistema de transacciones niega la transaccion indicando un bloqueo por malas practicas. De lo contrario, el sistema de transacciones consume el api de identificación de transacciones para determinar si se trata de malas practicas transaccionales

### **Proceso del servicio de identificación de malas practicas transaccionales**
1. El servicio tranaccional conoce toda la informacion de la transaccion actual: [merchant_id, _id, subsidiary, transaction_date, account_number, user_id, transaction_amount, transaction_type ] y envia esta información en tiempo real al API del modelo para saber si esta transaccion hace parte de malas preacticas transaccionales
4. El servicio del modelo primero realiza el calculo de las columnas adicionales (feature engineering) apoyado de la BD de historico de transacciónes (o al menos un delta de n dias / n=2 recomendado) y luego realiza la identificación de la transacción con el modelo
4. El modelo devuelve su respuesta y a su vez el servicio se lo devuelve al API Gateway. Por ultimo, este lo devuelve al servicio transaccional (app)
5. Finalmente, el servicio transaccional (app) decide si permite o no la transaccion y si genera una marca sobre el cliente con una marca de tiempo para futuras consultas (flujo ->Validacion marcación cliente con malas practicas transaccionales)







##### A tener encuenta:
- Si el cliente se marca con malas practicas en la BD de marcaciones el modelo es capaz de ignorar esta marcacion ya que este solo utiliza la información de la base de datos del historico de transacciones y solo en un rango de tiempo de 1 dia/24 horas. La marcacion es para apoyar el proceso transaccional cuando se haya detectado una vez en el dia las malas practicas transaccionales
- La BD Redis tiene que tener un TTL(time to live) para que los registros de marcacion del cliente se depuren una vez ya no apliquen, es decir, despues de 24 horas / 1 dia. Antes de borrarlos podria migrarse a otro tipo de almacenamiento para sacar insights importantes sobre comportamientos historicos de cada cliente
- El modelo  debe actualizarce (reentrenarse) diariamente o semanalmente con las nuevas transacciones para encontrar nuevos patrones que los clientes puedan estar implementando para ocultar las malas practicas transaccionales. La temporalidad se debe escoger de acuerdo con el riesgo que represente para el negocio la mala practica transaccional.
- Se propone comenzar con un modelo no supervisado ya que no se tiene etiqueta de cual transaccion reflejan malas practicas. Si el modelo no supervisado realiza correctamente la segmentación e identificación, se peude utilizar su salida para generar una nueva columna en la data con la clasificacion de la transaccion y asi poder entrenar un modelo supervisado basado en esa nueva información. Los errores que provea el modelo deben ser validados por humanos para garantizar que las etiquetas estan correctas. 